<a href="https://colab.research.google.com/github/kevinmcmanus/Astrophysics/blob/master/SagittariusA_Star.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sagittarius A*
Sagittarius A* is near the center of the Milky Way Galaxy. Therefore it's galactic longitude and latitude should be close to (0,0), we'll verify this. Also, we'll compute transit times for SagA* for the summer and winter solstices, to see if we're looking toward the center of the Milky Way in the summer or the winter.

In [0]:
# Python 3.x
from urllib.parse import urlencode
from urllib.request import urlretrieve

In [0]:
import numpy as np
import IPython.display

In [0]:
%matplotlib inline
from matplotlib import pyplot as plt

In [0]:
from astropy import units as u
from astropy.coordinates import SkyCoord
from astropy.table import Table

In [0]:
SagAStar = SkyCoord.from_name("Sagittarius A*")

In [6]:
SagAStar

<SkyCoord (ICRS): (ra, dec) in deg
    (266.41681663, -29.00782497)>

In [7]:
SagAStar.dec.to_string()

'-29d00m28.1699s'

In [0]:
#SagAStar.ra.hms.to_string()

In [8]:
'%d %d %f'%SagAStar.ra.hms + ' %d %d %f'%SagAStar.dec.dms

'17 45 40.035990 -29 0 -28.169899'

In [9]:
SagAStar.dec

<Latitude -29.00782497 deg>

If Sagittarius A* is at the center of the universe, its galactic coords should both be close to zero. Let's see.

In [10]:
SagAStar.galactic

<SkyCoord (Galactic): (l, b) in deg
    (359.94423568, -0.04616002)>

So not too far from (0,0)

## Observer Location

In [0]:
from astropy.coordinates import EarthLocation

In [12]:
# earth location for LTO
# LTO Berthoud CO
LTO_Latitude = 40.2994909
LTO_Longitude = -105.0848287
LTO_Elevation = 5049 #feet

#convert feet to meters
LTO_Elevation_m = (LTO_Elevation*u.imperial.ft).to(u.m)
LTO_Elevation_m

LTO = EarthLocation(lat=LTO_Latitude*u.degree, lon=LTO_Longitude*u.degree, height=LTO_Elevation_m)
LTO

<EarthLocation (-1268046.68608506, -4704542.52560169, 4104399.54327687) m>

In [13]:
LTO.lat


<Latitude 40.2994909 deg>

In [15]:
!pip install astroplan


    100% |████████████████████████████████| 276kB 10.3MB/s 
  Stored in directory: /root/.cache/pip/wheels/ea/4d/8e/765bbe02bf38e8bb2e7f400d30b1e1483c264f60776807fe2b
Successfully built astroplan


In [0]:
from astropy.time import Time
from astroplan import Observer

In [0]:
ltort = Observer(location=LTO, name="ltort", description = "Little Thomposon Observatory Radio Telescope",
                 timezone="US/Mountain")

In [18]:
ltort

<Observer: name='ltort',
    location (lon, lat, el)=(-105.0848287 deg, 40.2994909 deg, 1538.9351999992368 m),
    timezone=<DstTzInfo 'US/Mountain' LMT-1 day, 17:00:00 STD>>

In [0]:
winter_solstice = Time('2018-12-21 18:00') #6pm Berthoud time
summer_solstice = Time('2019-06-21 18:00')

Let's do M42 first. M42 is the Orion nebula and should be transiting sometime in the evening in the Winter sky.

In [0]:
#get coords for M42
M42 = SkyCoord.from_name('M42')

In [0]:
M42.dec.to_string()

'-5d23m27.996s'

In [0]:
'%d %d %f'%M42.ra.hms + ' %d %d %f'%M42.dec.dms

'5 35 17.299200 -5 -23 -27.996000'

In [0]:
M42_winter_transit = ltort.target_meridian_transit_time(winter_solstice, M42, which='next')
print("ISO: {0.iso}, JD: {0.jd}".format(M42_winter_transit)) 

ISO: 2018-12-22 06:33:29.496, JD: 2458474.773258053


In [0]:
#print(t.to_datetime(timezone=utc_plus_one_hour)) # to timezone-aware datetime
from datetime import datetime
from astropy.time import Time, TimezoneInfo
utc_minus_seven_hours = TimezoneInfo(utc_offset=-7*u.hour)
print(M42_winter_transit.to_datetime(timezone = utc_minus_seven_hours))

2018-12-21 23:33:29.495799-07:00


So M42 transits about 11:33 PM over LTO on December 21. Sounds right. Now for the summer transit:

In [0]:
M42_summer_transit = ltort.target_meridian_transit_time(summer_solstice, M42, which='next')
print(M42_summer_transit.to_datetime(timezone = utc_minus_seven_hours))

2019-06-21 11:37:52.903986-07:00


Roughly 11:37 AM (12:37 PM in June adjusting for DST) which also sounds about right. Now let'd do the same exercise for SagA*

In [0]:
SagAStar_winter_transit = ltort.target_meridian_transit_time(winter_solstice, SagAStar, which='next')
SagAStar_summer_transit = ltort.target_meridian_transit_time(summer_solstice, SagAStar, which='next')
print('Winter Transit:'+ str(SagAStar_winter_transit.to_datetime(timezone = utc_minus_seven_hours)))
print('Summer Transit:'+ str(SagAStar_summer_transit.to_datetime(timezone = utc_minus_seven_hours)))

Winter Transit:2018-12-21 11:46:01.638249-07:00
Summer Transit:2019-06-21 23:46:35.314430-07:00


This suggests that during the summer, looking due south at 11:30 PM (12:30 AM local time) we're looking at the center of the Milky Way

In [0]:
from astropy.coordinates import AltAz

aa = AltAz(location=LTO, obstime=SagAStar_summer_transit)
aa

<AltAz Frame (obstime=2458656.7823531763, location=(-1268046.686085056, -4704542.525601694, 4104399.5432768688) m, pressure=0.0 hPa, temperature=0.0 deg_C, relative_humidity=0, obswl=1.0 micron)>

Telescope coordinates. Azimuth should be close to 180 degrees.

In [0]:
SagAStar.transform_to(aa)

<SkyCoord (AltAz: obstime=2458656.7823531763, location=(-1268046.686085056, -4704542.525601694, 4104399.5432768688) m, pressure=0.0 hPa, temperature=0.0 deg_C, relative_humidity=0, obswl=1.0 micron): (az, alt) in deg
    ( 179.99993513,  20.68698533)>

For my 5 AM morning walks, say February 18, 2019:

In [0]:
walk_time = Time('2019-02-19 12:00') # needs to be in UTC (5am MST =12pm UTC)
aa = AltAz(location=LTO, obstime=walk_time)
aa

<AltAz Frame (obstime=2019-02-19 12:00:00.000, location=(-1268046.686085056, -4704542.525601694, 4104399.5432768688) m, pressure=0.0 hPa, temperature=0.0 deg_C, relative_humidity=0, obswl=1.0 micron)>

In [0]:
SagAStar.transform_to(aa)

<SkyCoord (AltAz: obstime=2019-02-19 12:00:00.000, location=(-1268046.686085056, -4704542.525601694, 4104399.5432768688) m, pressure=0.0 hPa, temperature=0.0 deg_C, relative_humidity=0, obswl=1.0 micron): (az, alt) in deg
    ( 142.98581997,  10.18632586)>

Approximately 10 degrees above the horizon in the southeast sky at 5 AM on February 18, 2019.

In [0]:
from astropy.coordinates import ICRS

In [0]:
LTO = EarthLocation(lat=LTO_Latitude*u.degree, lon=LTO_Longitude*u.degree, height=LTO_Elevation_m)
obs_time=Time('2018-08-05 19:08:13') # UT
aa = AltAz(location=LTO, obstime=obs_time,
          az = 180.3*u.degree,
          alt= 66.5*u.degree)
ltort_icrs = aa.transform_to(ICRS)

In [0]:
ltort_icrs.ra.hour, ltort_icrs.dec.degree

(9.058361991081979, 16.872688564694386)